### 가중 회귀분석(Weighted Regression)

In [ ]:
회귀 모델에서 모든 관측치가 동일한 중요도를 가지지 않을 때 사용하는 기법
특정 관측치에 더 많은 가중치를 주어, 그 관측치가 회귀 분석 결과에 더 큰 영향을 미치도록 합니다. 
이 방법은 데이터의 변동성이 다를 때, 즉 오차의 분산이 일정하지 않은 경우(이분산성)나 특정 관측치가 더 신뢰할 수 있을 때 유용

In [ ]:
# 가중 회귀분석의 원리
# 가중 회귀분석에서는 각 관측치에 가중치를 곱해줍니다. 
# 이 가중치는 해당 관측치의 중요도나 신뢰도를 반영합니다.
# 가중치가 높은 데이터 포인트는 회귀선을 결정하는 데 더 큰 영향을 미치며, 가중치가 낮은 데이터 포인트는 덜 영향을 미칩니다.

In [ ]:
# 사례: S&P 500 지수(SPY)와 애플(AAPL) 주식의 일일 수익률 분석
# 이 사례에서는 S&P 500 지수(SPY)를 시장 수익률의 대리 변수로 사용하고, 
# 애플(AAPL) 주식의 일일 수익률을 종속 변수로 사용하여 가중 회귀분석을 수행합니다. 
# 분석 기간은 2020년 1월 1일부터 2020년 12월 31일까지로 설정합니다.

In [ ]:
# 애플 주식의 일일 수익률을 종속 변수로, S&P 500 지수의 일일 수익률을 독립 변수로 사용하여 가중 회귀분석을 수행

#### 사례: S&P 500 지수(SPY)와 애플(AAPL) 주식의 일일 수익률 분석 (동일가중치 )

In [ ]:
# 동일 가중치 사례

In [1]:
import yfinance as yf
import pandas as pd
import statsmodels.api as sm

# 데이터 다운로드
spy = yf.download('SPY', start='2020-01-01', end='2020-12-31')
aapl = yf.download('AAPL', start='2020-01-01', end='2020-12-31')

# 일일 수익률 계산
spy_returns = spy['Adj Close'].pct_change().dropna()
aapl_returns = aapl['Adj Close'].pct_change().dropna()

# 데이터프레임 병합
data = pd.DataFrame({'SPY': spy_returns, 'AAPL': aapl_returns}).dropna()

# 가중치 설정 (예시로 모든 관측치에 동일한 가중치를 부여)
weights = pd.Series(1, index=data.index)

# 가중 회귀분석
X = sm.add_constant(data['SPY'])  # 독립변수에 상수항 추가
y = data['AAPL']  # 종속변수
wls_model = sm.WLS(y, X, weights=weights).fit()

# 결과 출력
print(wls_model.summary())


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                            WLS Regression Results                            
Dep. Variable:                   AAPL   R-squared:                       0.688
Model:                            WLS   Adj. R-squared:                  0.687
Method:                 Least Squares   F-statistic:                     551.5
Date:                Thu, 28 Mar 2024   Prob (F-statistic):           3.45e-65
Time:                        22:17:06   Log-Likelihood:                 678.14
No. Observations:                 252   AIC:                            -1352.
Df Residuals:                     250   BIC:                            -1345.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      

In [ ]:
# 결과 해석

In [ ]:
# 회귀 계수(coef), 
# : S&P 시장 수익률이 애플 주식의 수익률에 미치는 영향의 크기 --> 1.1584
# 통계적 유의성(P>|t|)
# : 0.000 < 0.05  통계적으로 유의미

#모델의 결정계수(R-squared)
# :  모델이 데이터의 변동성을 얼마나 잘 설명하는지? 약68% 설명력

#### 사례: S&P 500 지수(SPY)와 애플(AAPL) 주식의 일일 수익률 분석 (가중치 변경)

In [ ]:
#  np.linspace 함수를 사용하여 데이터 포인트에 선형적으로 증가하는 가중치를 부여 
# 분석 기간의 시작 부분에 있는 데이터 포인트에는 0.5의 가중치를, 
# 마지막 데이터 포인트에는 1.0의 가중치를 부여하여, 시간이 지남에 따라 데이터의 중요도가 증가하도록 설정. 
# 이러한 방식으로 가중치를 설정함으로써, 모델은 최근의 시장 동향을 더 잘 반영할 수 있다.

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 데이터 다운로드
spy = yf.download('SPY', start='2020-01-01', end='2020-12-31')
aapl = yf.download('AAPL', start='2020-01-01', end='2020-12-31')

# 일일 수익률 계산
spy_returns = spy['Adj Close'].pct_change().dropna()
aapl_returns = aapl['Adj Close'].pct_change().dropna()

# 데이터프레임 병합
data = pd.DataFrame({'SPY': spy_returns, 'AAPL': aapl_returns}).dropna()

# 가중치 설정 (최근 데이터에 더 높은 가중치 부여)
weights = np.linspace(start=0.5, stop=1.0, num=len(data))

# 가중 회귀분석
X = sm.add_constant(data['SPY'])  # 독립변수에 상수항 추가
y = data['AAPL']  # 종속변수
wls_model = sm.WLS(y, X, weights=weights).fit()

# 결과 출력
print(wls_model.summary())


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                            WLS Regression Results                            
Dep. Variable:                   AAPL   R-squared:                       0.651
Model:                            WLS   Adj. R-squared:                  0.649
Method:                 Least Squares   F-statistic:                     465.4
Date:                Thu, 28 Mar 2024   Prob (F-statistic):           5.26e-59
Time:                        22:27:07   Log-Likelihood:                 670.19
No. Observations:                 252   AIC:                            -1336.
Df Residuals:                     250   BIC:                            -1329.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      

In [ ]:
# 결과 해석

In [ ]:
# 가중치를 변경함으로써, 모델은 최근 데이터의 패턴을 더 중요하게 고려하게 되며, 
# 이는 회귀계수의 추정치와 모델의 전반적인 성능에 영향을 미칠 수 있다.
# 회귀계수 변화 : 1.1584 -->  1.1725 
# P-value는 회귀 계수의 통계적 유의성을 평가 : 0.000 < 0.05 통계적 유의

### 사례: S&P 500 지수(SPY)와 애플(AAPL) 주식의 일일 수익률 분석 (OLS 사용)

In [4]:
import yfinance as yf
import pandas as pd
import statsmodels.api as sm

# 데이터 다운로드
spy = yf.download('SPY', start='2020-01-01', end='2020-12-31')
aapl = yf.download('AAPL', start='2020-01-01', end='2020-12-31')

# 일일 수익률 계산
spy_returns = spy['Adj Close'].pct_change().dropna()
aapl_returns = aapl['Adj Close'].pct_change().dropna()

# 데이터프레임 병합
data = pd.DataFrame({'SPY': spy_returns, 'AAPL': aapl_returns}).dropna()

# OLS 회귀분석
X = sm.add_constant(data['SPY'])  # 독립변수에 상수항 추가
y = data['AAPL']  # 종속변수
ols_model = sm.OLS(y, X).fit()

# 결과 출력
print(ols_model.summary())


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                            OLS Regression Results                            
Dep. Variable:                   AAPL   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.687
Method:                 Least Squares   F-statistic:                     551.5
Date:                Thu, 28 Mar 2024   Prob (F-statistic):           3.45e-65
Time:                        22:24:19   Log-Likelihood:                 678.14
No. Observations:                 252   AIC:                            -1352.
Df Residuals:                     250   BIC:                            -1345.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      

In [ ]:
# 결과해석
# 가중회귀분석과 비교 -- 동일가중치